In [41]:
import asyncio
from typing import List
import httpx
import json
from parsel import Selector
import pandas as pd

In [35]:
async def scrape_properties(urls: List[str], client):

    to_scrape = [client.get(url) for url in urls]
    results = []

    for response in asyncio.as_completed(to_scrape):
        response = await response
        assert response.status_code == 200, "request has been blocked"
        selector = Selector(response.text)
        data = selector.css("script#__NEXT_DATA__::text").get()
        if data:
            # Option 1: some properties are located in NEXT DATA cache
            data = json.loads(data)

            results_temp = data["props"]["pageProps"]["searchPageState"]["cat1"]["searchResults"]["listResults"]
            results.extend(results_temp)
    
    return results

In [36]:
client = httpx.AsyncClient(
    # enable http2
    http2=True,
    # add basic browser like headers to prevent being blocked
    headers={
        "accept-language": "en-US,en;q=0.9",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "accept-language": "en-US;en;q=0.9",
        "accept-encoding": "gzip, deflate, br",
    },
)

urls = ['https://www.zillow.com/los-angeles-ca/']

In [38]:
results = await scrape_properties(urls, client)

In [42]:
df = pd.DataFrame(results)
df

,zpid,id,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,countryCurrency,...,list,relaxed,brokerName,carouselPhotos,zestimate,info6String,hasOpenHouse,openHouseStartDate,openHouseEndDate,openHouseDescription
0,19921550,19921550,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/b58d7b6465e...,True,https://www.zillow.com/homedetails/19120-Besse...,FOR_SALE,House for sale,$,...,True,False,"Pinnacle Estate Properties, Inc.",[{'url': 'https://photos.zillowstatic.com/fp/b...,NaN,NaN,NaN,NaN,NaN,NaN
1,20092286,20092286,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/07c56aa312f...,True,https://www.zillow.com/homedetails/7841-Wentwo...,FOR_SALE,House for sale,$,...,True,False,Berkshire Hathaway HomeServices California Pro...,[{'url': 'https://photos.zillowstatic.com/fp/0...,433100.0,Nancy Sanborn DRE # 00771096,NaN,NaN,NaN,NaN
2,19922408,19922408,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/a58539fc55f...,True,https://www.zillow.com/homedetails/19140-Archw...,FOR_SALE,House for sale,$,...,True,False,The Agency,[{'url': 'https://photos.zillowstatic.com/fp/a...,NaN,NaN,True,2024-04-04T17:00:00,2024-04-04T20:00:00,Open House - 5:00 - 8:00 PM
3,20183754,20183754,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/ff76e7f16cb...,True,https://www.zillow.com/homedetails/8510-Wilbur...,FOR_SALE,House for sale,$,...,True,False,Equity Union,[{'url': 'https://photos.zillowstatic.com/fp/f...,628600.0,NaN,NaN,NaN,NaN,NaN
4,20803593,20803593,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/9ca54057aab...,True,https://www.zillow.com/homedetails/1851-Outpos...,FOR_SALE,House for sale,$,...,True,False,The Flying V Realty Inv. Co.,[{'url': 'https://photos.zillowstatic.com/fp/9...,3070100.0,Robert Valandra DRE # 01067887,NaN,NaN,NaN,NaN
5,19991985,19991985,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/f127e199701...,True,https://www.zillow.com/homedetails/4146-Meadow...,FOR_SALE,House for sale,$,...,True,False,The Agency,[{'url': 'https://photos.zillowstatic.com/fp/f...,3126500.0,Jon Grauman DRE # 01469825,NaN,NaN,NaN,NaN
6,20538291,20538291,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/7b231bccc34...,True,https://www.zillow.com/homedetails/13011-W-Sun...,FOR_SALE,House for sale,$,...,True,False,Berkshire Hathaway HomeServices California Pro...,[{'url': 'https://photos.zillowstatic.com/fp/7...,3533800.0,David Offer DRE # 01150357,True,2024-04-02T11:00:00,2024-04-02T14:00:00,Open House - 11:00 AM - 2:00 PM
7,345484490,345484490,ForSale,New Construction,https://photos.zillowstatic.com/fp/0284cf0025e...,True,https://www.zillow.com/homedetails/443-W-Avenu...,FOR_SALE,New construction,$,...,True,False,Keller Williams Realty Los Feliz,[{'url': 'https://photos.zillowstatic.com/fp/0...,NaN,Kristopher Edwards DRE # 02134805,NaN,NaN,NaN,NaN
8,20449540,20449540,ForSale,RecentChange,https://photos.zillowstatic.com/fp/a73688cb746...,True,https://www.zillow.com/homedetails/2543-Walnut...,FOR_SALE,Auction,$,...,True,False,Coastal Premier Properties,[{'url': 'https://photos.zillowstatic.com/fp/a...,NaN,NaN,NaN,NaN,NaN,NaN
9,20633098,20633098,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/5de1b45d494...,True,https://www.zillow.com/homedetails/720-Penrith...,FOR_SALE,House for sale,$,...,True,False,The Network Agency,[{'url': 'https://photos.zillowstatic.com/fp/5...,587300.0,NaN,NaN,NaN,NaN,NaN


In [43]:
df.to_csv("LA.csv", index=False)